In [ ]:
import pathlib
# import xml.etree.ElementTree

import lxml.etree as ET

import dlsproc.xml

Directory where the data (*XML* files) are stored

In [ ]:
directory = pathlib.Path.home() / 'xml_parser/sample_files/PlataformasAgregadasSinMenores_202201/'
assert directory.exists()

A (sample) file in that directory

In [ ]:
input_file = directory / 'PlataformasAgregadasSinMenores_20220104_030016_1.atom'
assert input_file.exists()
input_file

PosixPath('/home/manu/xml_parser/sample_files/PlataformasAgregadasSinMenores_202201/PlataformasAgregadasSinMenores_20220104_030016_1.atom')

In [ ]:
dlsproc.xml.get_namespaces(input_file)

{'cbc-place-ext': 'urn:dgpe:names:draft:codice-place-ext:schema:xsd:CommonBasicComponents-2',
 'cac-place-ext': 'urn:dgpe:names:draft:codice-place-ext:schema:xsd:CommonAggregateComponents-2',
 'cbc': 'urn:dgpe:names:draft:codice:schema:xsd:CommonBasicComponents-2',
 'cac': 'urn:dgpe:names:draft:codice:schema:xsd:CommonAggregateComponents-2',
 'ns1': 'urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2',
 'base': 'http://www.w3.org/2005/Atom'}